In [ ]:
# 使用新的配置模块进行优化实验
from optuna_config import create_robust_study, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 方法1: 创建新的研究
study_name = "backtest_tuning_with_side_auc_and_meta_f1"  # 固定名称，便于后续继续

with optuna_log_manager.verbose_optimization():
    study = create_robust_study(
        study_name=study_name,
        storage_dir="optuna_storage",
        direction="maximize",
        n_startup_trials=5000,
        percentile_for_pruning=20.0,
    )

    # 使用安全优化函数
    safe_optimize(
        study=study,
        objective=tune_pipeline,
        n_trials=25000,
        n_jobs=1,
        gc_after_trial=True,
        show_progress_bar=False,
    )

[I 2025-06-15 12:34:47,044] A new study created in RDB with name: backtest_tuning_with_side_auc_and_meta_f1
INFO:optuna_config:研究 'backtest_tuning_with_side_auc_and_meta_f1' 已加载
INFO:optuna_config:数据库路径: optuna_storage/optuna_study.db
INFO:optuna_config:已完成试验数: 0
INFO:optuna_config:开始优化，目标试验数: 25000
INFO:optuna_config:当前已完成试验数: 0
[I 2025-06-15 12:46:07,903] Trial 0 finished with value: -100.0 and parameters: {'n1': 89, 'n2': 23, 'n_entropy': 67, 'threshold': 0.34264745725192597}. Best is trial 0 with value: -100.0.
[I 2025-06-15 12:53:53,844] Trial 1 finished with value: -100.0 and parameters: {'n1': 246, 'n2': 63, 'n_entropy': 279, 'threshold': 1.3716111113250034}. Best is trial 0 with value: -100.0.
[I 2025-06-15 12:59:14,669] Trial 2 finished with value: -100.0 and parameters: {'n1': 182, 'n2': 93, 'n_entropy': 62, 'threshold': 1.4372264880557297}. Best is trial 0 with value: -100.0.
[I 2025-06-15 13:04:58,718] Trial 3 finished with value: -100.0 and parameters: {'n1': 30, 'n2': 293

In [3]:
sorted(study.trials, key=lambda t: t.value if t.value is not None else 0, reverse=True)[
    :3
]

[FrozenTrial(number=429, state=1, values=[211.10774416822937], datetime_start=datetime.datetime(2025, 6, 13, 3, 33, 9, 541559), datetime_complete=datetime.datetime(2025, 6, 13, 3, 48, 13, 565337), params={'n1': 48, 'n2': 93, 'n_entropy': 190, 'threshold': 0.11103525552108519}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n1': IntDistribution(high=300, log=False, low=1, step=1), 'n2': IntDistribution(high=300, log=False, low=1, step=1), 'n_entropy': IntDistribution(high=300, log=False, low=30, step=1), 'threshold': FloatDistribution(high=1.086986528322778, log=False, low=0.09880863025791121, step=None)}, trial_id=430, value=None),
 FrozenTrial(number=406, state=1, values=[174.95013862683962], datetime_start=datetime.datetime(2025, 6, 12, 22, 1, 34, 658938), datetime_complete=datetime.datetime(2025, 6, 12, 22, 17, 56, 911547), params={'n1': 65, 'n2': 77, 'n_entropy': 237, 'threshold': 0.1143095516135105}, user_attrs={}, system_attrs={}, intermediate_values={}, 

In [ ]:
# 方法2: 继续之前中断的实验
from optuna_config import OptunaStorageManager, safe_optimize
from tuning_pipeline import optuna_log_manager, tune_pipeline

# 使用存储管理器
manager = OptunaStorageManager("optuna_storage")

# 列出所有可用的研究
studies = manager.list_studies()
if studies:
    print("可用的研究:")
    for i, summary in enumerate(studies):
        print(f"  {i + 1}. {summary.study_name}: {summary.n_trials} 个试验")

        # 方法2a: 继续特定的研究
        study_name_to_continue = "backtest_tuning_main"  # 指定要继续的研究名称

        if manager.study_exists(study_name_to_continue):
            print(f"\n继续研究: {study_name_to_continue}")

            with optuna_log_manager.verbose_optimization():
                # 直接加载指定的研究
                study = manager.create_or_load_study(
                    study_name=study_name_to_continue, load_if_exists=True
                )

                # 显示当前状态
                info = manager.get_study_info(study_name_to_continue)
                print(f"研究信息: {info}")

                # 继续优化
                safe_optimize(
                    study=study,
                    objective=tune_pipeline,
                    n_trials=50000,
                    n_jobs=1,
                    gc_after_trial=True,
                )
        else:
            print(f"研究 '{study_name_to_continue}' 不存在")

        # 方法2b: 继续最新的研究
        # latest_study = manager.get_latest_study()
        # if latest_study:
        #     print(f"继续最新的研究: {latest_study.study_name}")

        #     with optuna_log_manager.verbose_optimization():
        #         safe_optimize(
        #             study=latest_study,
        #             objective=tune_pipeline,
        #             n_trials=50000,
        #             n_jobs=1,
        #             gc_after_trial=True,
        #         )
        # else:
        #     print("没有找到任何研究，请先运行第一个cell创建新研究")
else:
    print("没有找到任何研究，请先运行第一个cell创建新研究")